In [ ]:
from summarizer import TransformerSummarizer

## GPT2

gpt_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

# Function to summarize an article
def summarize_text(article, model, min_length=50, max_length=150):
    try:
        # Tokenize and process on GPU
        summary = ''.join(model(article, min_length=min_length, max_length=max_length))
        return summary
    except Exception as e:
        return str(e)


file_path = "/content/reference_news_data_with_summaries (2).csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Apply summarization to each article
data['Summary_gpt'] = data['Cleaned Article Text'].apply(lambda x: summarize_text(x, gpt_model))

# Save the summarized data
output_file_path = "/content/reference_news_data_with_summaries (2).csv"
data.to_csv(output_file_path, index=False)
print(f"Summarized file saved at: {output_file_path}")

Summarized file saved at: /content/reference_news_data_with_summaries (2).csv


In [2]:
!pip install bert-extractive-summarizer

In [ ]:
pip install rouge-score nltk pandas

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ac0152ce018ce998e0cbb56c2f832af96df2a69f8a4b9872bfd97e9a7a954134
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the dataset
reference_file_path = "/content/data_with_summaries.csv"  # Replace with your file path
data = pd.read_csv(reference_file_path)

# Define the reference column and T5-generated summary columns
reference_column = "Summary"  # Reference summaries
generated_columns = ["Summary_gpt"]  # T5 model summaries

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    """
    Calculate ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    if pd.isnull(reference) or pd.isnull(generated):
        return 0.0, 0.0, 0.0  # Handle missing values gracefully

    scores = scorer.score(reference, generated)
    rouge1 = scores['rouge1'].fmeasure
    rouge2 = scores['rouge2'].fmeasure
    rougeL = scores['rougeL'].fmeasure

    return rouge1, rouge2, rougeL

# Process each T5 model column
for gen_col in generated_columns:
    if gen_col in data.columns:
        print(f"Calculating ROUGE scores for {gen_col}")
        scores = data.apply(lambda row: calculate_rouge(row[reference_column], row[gen_col]), axis=1)

        # Add new columns for each ROUGE score
        data[f'ROUGE-1_{gen_col}'] = scores.apply(lambda x: x[0])
        data[f'ROUGE-2_{gen_col}'] = scores.apply(lambda x: x[1])
        data[f'ROUGE-L_{gen_col}'] = scores.apply(lambda x: x[2])
    else:
        print(f"Column {gen_col} not found in the dataset.")

# Save the results to a new CSV file
output_file_path = "news_data_with_rouge_scores.csv"
data.to_csv(output_file_path, index=False)

print(f"ROUGE scores saved to: {output_file_path}")



Calculating ROUGE scores for Summary_gpt
ROUGE scores saved to: news_data_with_rouge_scores.csv


In [3]:
from summarizer import TransformerSummarizer
from transformers import AutoModel, AutoTokenizer

# Specify GPT-2 model
model_name = "gpt2-medium"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Save model and tokenizer
model_save_path = '/content/gpt2-medium-model'
tokenizer_save_path = '/content/gpt2-medium-tokenizer'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to: {model_save_path}")
print(f"Tokenizer saved to: {tokenizer_save_path}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Model saved to: /content/gpt2-medium-model
Tokenizer saved to: /content/gpt2-medium-tokenizer
